In [39]:
import pandas as pd
import requests

In [40]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'
runFiles  = ['r1-3389','r2-3869','r3-4283','r4-4190','r5-3503','r6-3598',
             'r7-4372','r8-3473','r9-3858','r10-3787','r11-3797','r12-3798',
             'r13-12260','r14-8393','r15-3769','r16p-1227','r17-4041','r18p-347',
             'r19p-288','r20p-2944']

In [41]:
!ls -lrt /Users/amcguire/Documents/PERSONAL/KOL/raidLogs/

total 856
-rw-r--r--@ 1 amcguire  staff  12204 Jan 10 21:03 r1-3389.txt
-rw-r--r--@ 1 amcguire  staff  17303 Jan 10 21:04 r2-3869.txt
-rw-r--r--@ 1 amcguire  staff  16436 Jan 10 21:05 r3-4283.txt
-rw-r--r--@ 1 amcguire  staff  14890 Jan 14 19:39 r4-4190.txt
-rw-r--r--@ 1 amcguire  staff  13289 Jan 14 19:40 r5p-2971.txt
-rw-r--r--@ 1 amcguire  staff  10060 Jan 14 19:41 r6p-2980.txt
-rw-r--r--@ 1 amcguire  staff  13456 Jan 21 11:16 r7p-3734.txt
-rw-r--r--@ 1 amcguire  staff  16206 Jan 21 11:18 r5-3503.txt
-rw-r--r--@ 1 amcguire  staff   9933 Jan 21 11:19 r9p-2785.txt
-rw-r--r--@ 1 amcguire  staff  12983 Jan 21 11:19 r6-3598.txt
-rw-r--r--@ 1 amcguire  staff  15787 Jan 23 20:01 r7-4372.txt
-rw-r--r--@ 1 amcguire  staff   6053 Jan 23 20:03 r11p-1691.txt
-rw-r--r--@ 1 amcguire  staff  15167 Jan 23 20:03 r9-3858.txt
-rw-r--r--@ 1 amcguire  staff  12888 Jan 29 17:21 r8p-2877.txt
-rw-r--r--@ 1 amcguire  staff  10956 Jan 29 17:22 r10p-2841.txt
-rw-r--r--@ 1 amcguire  staff  15407 Jan 29 17:24 r

In [42]:
# Populate our run dictionary
allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [43]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
            
collectRaiders(allRuns)

['themadowl',
 'Liora',
 'Volotani',
 'kenny kamAKAzi',
 'AeNimUS',
 'kirByllAmA',
 'gregmasta',
 'HeroBill',
 'Gausie',
 'tdog21',
 'madowl',
 'Phillammon',
 'Name Guy Man',
 'NukebOy1313',
 'chameco',
 'John Keel',
 'RandomExtremity',
 'Just Eyes',
 'Lyft',
 'Arkcon',
 'threebullethamburgler',
 'violetinsane',
 'Captain Scotch',
 'Stary']

In [44]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if row[0:len(name)] == name: # fixing the madowl bug ;-;
                if 'was defeated' in row:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    killed += tSpent
                else:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [45]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})
            
tabulateLoots('Captain Scotch', allRuns)

{'loot': ["Drunkula's wineglass",
  'zombie accordion',
  "Mayor Ghost's cloak",
  "Drunkula's bell",
  "Mayor Ghost's toupee",
  "Great Wolf's headband",
  "Mayor Ghost's sash",
  "Great Wolf's headband",
  "Unkillable Skeleton's restless leg",
  "Unkillable Skeleton's restless leg",
  'Quiets-Your-Steps',
  'Gets-You-Drunk',
  'HOA zombie eyes'],
 'skills': 7}

In [47]:
finalDisplay = {}

for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])
    
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot']]
finalFrame['kill/skillz'] = finalFrame['kills']/(finalFrame['skills']+0.5)

finalFrame.sort_values(by='kill/skillz', ascending=False)

,kills,defeats,skills,lootCount,loot,kill/skillz
themadowl,5142,66,1,10,"[Drunkula's bell, Covers-Your-Head, Drunkula's...",3428.000000
kenny kamAKAzi,1309,1,0,0,[],2618.000000
Captain Scotch,9370,33,7,13,"[Drunkula's wineglass, zombie accordion, Mayor...",1249.333333
gregmasta,2982,25,2,10,"[Gets-You-Drunk, Unkillable Skeleton's shingua...",1192.800000
Arkcon,554,8,0,0,[],1108.000000
Liora,548,7,0,1,[Great Wolf's left paw],1096.000000
Volotani,2653,4,2,5,"[Mayor Ghost's khakis, Mayor Ghost's scissors,...",1061.200000
kirByllAmA,3541,4,3,1,[Gets-You-Drunk],1011.714286
Just Eyes,5366,63,5,5,"[Gets-You-Drunk, Drunkula's wineglass, Protect...",975.636364
chameco,4197,0,4,1,[Gets-You-Drunk],932.666667


In [48]:
print("This effort has gotten {} skills for {} Loathers. Good job folks!".format(finalFrame['skills'].sum(),len(finalFrame['skills'])))

This effort has gotten 48 skills for 24 Loathers. Good job folks!


In [49]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))

In [10]:
n = "Liora"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 0, 'defeats': 0}
r2-3869 - {'kills': 0, 'defeats': 0}
r3-4283 - {'kills': 548, 'defeats': 7}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 0, 'defeats': 0}
r6-3598 - {'kills': 0, 'defeats': 0}
r7-4372 - {'kills': 0, 'defeats': 0}
r8-3473 - {'kills': 0, 'defeats': 0}
r9-3858 - {'kills': 0, 'defeats': 0}
r10-3787 - {'kills': 0, 'defeats': 0}
r11-3797 - {'kills': 0, 'defeats': 0}
r12-3798 - {'kills': 0, 'defeats': 0}
r13-12260 - {'kills': 0, 'defeats': 0}
r14p-5188 - {'kills': 0, 'defeats': 0}
r15-3769 - {'kills': 0, 'defeats': 0}
r16p-1227 - {'kills': 0, 'defeats': 0}
r17p-779 - {'kills': 0, 'defeats': 0}
r18p-347 - {'kills': 0, 'defeats': 0}


In [11]:
n = "gregmasta"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 0, 'defeats': 0}
r2-3869 - {'kills': 0, 'defeats': 0}
r3-4283 - {'kills': 0, 'defeats': 0}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 0, 'defeats': 0}
r6-3598 - {'kills': 0, 'defeats': 0}
r7-4372 - {'kills': 0, 'defeats': 0}
r8-3473 - {'kills': 0, 'defeats': 0}
r9-3858 - {'kills': 1, 'defeats': 0}
r10-3787 - {'kills': 0, 'defeats': 0}
r11-3797 - {'kills': 115, 'defeats': 0}
r12-3798 - {'kills': 361, 'defeats': 0}
r13-12260 - {'kills': 752, 'defeats': 5}
r14p-5188 - {'kills': 744, 'defeats': 0}
r15-3769 - {'kills': 688, 'defeats': 7}
r16p-1227 - {'kills': 0, 'defeats': 0}
r17p-779 - {'kills': 321, 'defeats': 13}
r18p-347 - {'kills': 0, 'defeats': 0}
